In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
from urllib.parse import quote
from logging import exception
from tqdm import tqdm
import pandas as pd
import numpy as np
import platform
import time
import click
import sys
import os
import re

In [ ]:
df = pd.read_csv("시연이미지_total.csv")
df = df.sample(100)
del df['Unnamed: 0']
df = df.reset_index(drop=True)
df

,이미지id,url,src
0,9023592.jpg,/p/CgN4DkIpM6F/,https://scontent-gmp1-1.cdninstagram.com/v/t51...
1,9010120.jpg,/p/CgLqsVDPhE4/,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,9007088.jpg,/p/CgLsn5IOm-j/,https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,9002184.jpg,/p/CeUih8jBDWS/,https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,9024479.jpg,/p/CgN6212J1tl/,https://scontent-gmp1-1.cdninstagram.com/v/t51...
...,...,...,...
1995,9007635.jpg,/p/CgL5aSAvVci/,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1996,9021073.jpg,/p/CgLO7nAPZgj/,https://scontent-gmp1-1.cdninstagram.com/v/t51...
1997,9001836.jpg,/p/CgLatTcL_0k/,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1998,9018688.jpg,/p/CgMktXoPVUF/,https://scontent-gmp1-1.cdninstagram.com/v/t51...


#### 로그인

In [ ]:
driver = Chrome("c:/pydata/chromedriver.exe")
url =" https://www.instagram.com/"
uid = ''
upw = ''
driver.get(url)
time.sleep(2)
driver.find_element(By.NAME, "username").send_keys(uid)
driver.find_element(By.NAME, "password").send_keys(upw)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()

time.sleep(4)


C:\Users\CJ\AppData\Local\Temp\ipykernel_21848\2149207174.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("c:/pydata/chromedriver.exe")


#### 크롤링 파트

In [ ]:
# cloth = pd.read_csv("C:/Users/CJ/Desktop/MultiCampus/프로젝트_3_마지막/크롤링/csv/옷스타그램_id_url_src_0707.csv")
content_lst =[]
reply_lst = []
like_lst =[]
for j in tqdm(df.index):
    num =  j
    im_url = df.iloc[num:,-2]
    im_url = "https://www.instagram.com" + im_url.values[0]
    driver.get(im_url)
    time.sleep(3)
    
    # 게시글 크롤링 및 더보기 버튼 유무 확인
    try:    
        html = driver.page_source
        soup = bs(html, "html.parser")
        content = soup.find("div", class_="_a9zs").getText()
        img_name = df.iloc[num,0]
        content_lst.append({img_name:content})
        # 좋아요 가져오기
        if soup.find("div","_aacl _aaco _aacw _aacx _aada _aade").get_text() == "여러 명":
            like_lst.append({img_name:-1})
        else :
            like_lst.append({img_name:soup.find("div","_aacl _aaco _aacw _aacx _aada _aade").find("span").get_text()})
        # 더 보기 버튼 몇개 있는지 확인
        button_lst = soup.find("ul",class_="_a9z6 _a9za").find_all('button', class_="_acan _acao _acas")
        text = soup.find("ul", class_="_a9z6 _a9za").find_all("li")[-1].get_text()
        plus_button = soup.find("ul", class_="_a9z6 _a9za").find_all("li")[-1].find_all("button")
        if len(plus_button[0].find_all("svg")) == 0: plus_button = []
        elif plus_button[0].find_all("svg") != '댓글 더 읽어들이기' : plus_button = []
    except Exception as e:
        print(str(num)+ " : url오류")

        
    # 댓글 더 보기 확인
    cnt = 0 # 4번 이상 더 누르면 그만
    while len(plus_button) != 0 :
        if cnt > 4 : break
        if plus_button == 0 : break;
        driver.find_element(By.CSS_SELECTOR, 'div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p > div > div._aasx > div._aat6 > ul > li > div > button').send_keys(Keys.ENTER)
        time.sleep(2)
        html = driver.page_source
        soup = bs(html, "html.parser")
        plus_button = soup.find("ul", class_="_a9z6 _a9za").find_all("li")[-1].find_all("button")
        if len(plus_button[0].find_all("svg")) == 0: break
        elif plus_button[0].find_all("svg") != '댓글 더 읽어들이기' : break
        cnt += 1
    time.sleep(1)
    
    # 답글 더 보기 확인
    while len(button_lst) != 0:
        start = 2; end = len(button_lst)+2
        for i in range(start, end):
            if len(button_lst) > 1:
                n = str(i)
                try:
                    driver.find_element(By.CSS_SELECTOR, f'div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p._abcm > div > div._aasx > div._aat6 > ul > ul:nth-child({n}) > li > ul > li > div > button').send_keys(Keys.ENTER)
                except:                                   
                    print("error")
            else:
                driver.find_element(By.CSS_SELECTOR, f'div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div._a3gq._ab-1 > section > main > div._aa6b._aa6d > div._aa6e > article > div > div._ab8w._ab94._ab99._ab9f._ab9m._ab9p._abcm > div > div._aasx > div._aat6 > ul > ul > li > ul > li > div > button').send_keys(Keys.ENTER)
        html = driver.page_source
        soup = bs(html, "html.parser")
        buttons = soup.find("ul",class_="_a9ym").find_all('button', class_="_acan _acao _acas")
        button_lst = [button for button in buttons if "답글 보기" in button.get_text()]
    time.sleep(1)

    # 댓글 크롤링
    html = driver.page_source
    soup = bs(html, "html.parser")
    ul_lst = soup.find_all("ul", "_a9ym")
    ul_lsts = []
    for ul in ul_lst:
        for span in ul.find_all("span", "_aacl _aaco _aacu _aacx _aad7 _aade"):
            ul_lsts.append(span.get_text())
    reply_lst.append({img_name:ul_lsts})

  0%|          | 2/2000 [00:12<3:20:58,  6.04s/it]

2 : url오류


  0%|          | 9/2000 [00:57<3:24:41,  6.17s/it]

9 : url오류


  0%|          | 10/2000 [01:02<3:20:19,  6.04s/it]

...

274 : url오류


 14%|█▍        | 275/2000 [27:17<2:34:02,  5.36s/it]

275 : url오류


 14%|█▍        | 276/2000 [27:26<2:51:22,  5.96s/it]


KeyboardInterrupt: 

In [ ]:
import os
if not os.path.exists("./시연크롤링"):
    os.mkdir("시연크롤링")

In [ ]:
lst_dir = os.listdir("시연크롤링/")
cnt=1
while(True):
    if f"게시글{cnt}.csv" not in lst_dir:
        pd.DataFrame(content_lst).to_csv(f"시연크롤링/게시글{cnt}.csv")
        pd.DataFrame(content_lst).to_csv(f"시연크롤링/댓글{cnt}.csv")
        pd.DataFrame(content_lst).to_csv(f"시연크롤링/좋아요{cnt}.csv")
        break
    cnt+=1

['게시글1.csv',
 '게시글2.csv',
 '게시글3.csv',
 '게시글4.csv',
 '댓글1.csv',
 '댓글2.csv',
 '댓글3.csv',
 '댓글4.csv',
 '좋아요1.csv',
 '좋아요2.csv',
 '좋아요3.csv',
 '좋아요4.csv']